In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
sub_LO = pd.read_csv("../Data/sub_LO.csv")


# London

In [3]:
London = 51.505024,-0.080416

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about

In [6]:
starbucks_Lo = fun.get_results("starbucks", London, 10, 1000, token_foursquare)
df_Lo = fun.clean_results_first_lo(starbucks_Lo, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_Lo = fun.get_results("Nursery school", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results_lo(kinder_Lo, df_Lo, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [8]:
club_Lo = fun.get_results("night club", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results(club_Lo, df_Lo, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_Lo = fun.get_results("vegan", London, 10, 3000, token_foursquare)
df_Lo = fun.clean_results_lo(vegan_Lo, df_Lo, "Vegan restaurant")

- We look for dog hairdressers restaurants around the office and clean the results keeping only the columns that we are interested about

In [10]:
pet_Lo = fun.get_results("pet grooming", London, 5, 3000, token_foursquare)
df_Lo = fun.clean_results(pet_Lo, df_Lo, "Pet hairdresser")

- We look for a basketball arena. However, only local UK games are playing nowadays in London. 

In [11]:
Basketball_Lo = fun.get_results("Basketball arena", London, 1, 5000, token_foursquare)
# df_Lo = fun.clean_results_lo(Basketball_Lo, df_Lo, "Basketball arena")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_Lo = fun.get_results_airport("airport", London, 3, 50000, token_foursquare)
df_Lo = fun.clean_results_lo(Airports_Lo, df_Lo, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_Lo = fun.add_company(df_Lo, "1-2 More London Pl, London", "Southwark", 51.505024, -0.080416)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_Lo["Distance"] = df_Lo.apply(fun.calc_distance_LO, axis = 1).round(2)

In [15]:
df_Lo

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"1-2 More London Pl, London",Southwark,51.505024,-0.080416,NaN,0.00
1,Starbucks,"49 Shad Thames, London, Greater London, SE1 2NJ",L,51.504041,-0.075553,Coffee,354.93
2,Starbucks,"The Shard Arcade (London Bridge St), London, G...",L,51.505072,-0.087198,Coffee,470.92
3,Starbucks,"54 Tooley St, Bridge, Greater London, SE1 9SP",L,51.505487,-0.085133,Coffee,331.54
4,Starbucks,"11 Leadenhall St, London, Greater London, EC3V...",L,51.513328,-0.082907,Coffee,939.93
5,Starbucks,"2A King William House, London, Greater London,...",L,51.510546,-0.085579,Coffee,711.29
6,Starbucks,"3 Tower Pl (East Building), London, Greater Lo...",L,51.508576,-0.079207,Coffee,404.00
7,Starbucks,"London, Greater London, EC3A 4AX",L,51.513846,-0.081034,Coffee,982.46
8,Starbucks,"48-50 Minories, London, Greater London, EC3N 1JJ",L,51.511469,-0.075064,Coffee,807.62
9,Starbucks,"St. Katherine's Way (St. Katherines Way), Lond...",L,51.506661,-0.073780,Coffee,495.44


In [16]:
Lo_map = Map(location = London, zoom_start = 12)

In [17]:
for index, row in df_Lo.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
            
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(Lo_map)

In [19]:
for index, row in sub_LO.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(Lo_map)

In [20]:
Lo_map

In [21]:
Lo_map.save('../images/London_map.html')